In [92]:
import pandas as pd
import geopandas as gpd
import geocoder
import numpy as np
from shapely.geometry import Point

Election districts

In [128]:
ed = gpd.read_file('day3/Election Districts.geojson')

In [173]:
ed['elect_dist_new'] = ed.elect_dist.apply(lambda x: int(x[2:]))

Election District to Pollsite table

In [107]:
ed_pollsite = pd.read_csv('day3/Election_District_Poll_Sites.csv')

In [264]:
site_ED  = ed_pollsite.groupby('SITE_NUMBER')['ED'].apply(list)

In [135]:
ed1 = ed.merge(ed_pollsite,left_on='elect_dist_new',right_on='ED')

Pollsites

In [4]:
pollsites = pd.read_csv('day3/Voting_Poll_Sites.csv')

In [22]:
pollsites['STREET_NUMBER_new'] = pollsites['STREET_NUMBER'].apply(lambda x: x.split(".")[0])

In [26]:
pollsites['VOTER_ENTRANCE_new'] = pollsites.apply(lambda x: x['STREET_NUMBER_new']+" "+x['STREET_NAME']+" "+x['CITY']+" "+x['POSTCODE'],axis=1)
pollsites['HANDICAP_ENTRANCE_new'] = pollsites.apply(lambda x: x['HANDICAP_ENTRANCE']+" "+x['CITY']+" "+x['POSTCODE'],axis=1)

In [51]:
pollsites['latlng'] = pollsites.apply(lambda x: geocoder.mapbox(x['VOTER_ENTRANCE_new'],
                                                                key='INSERT KEY').latlng,axis=1)
pollsites['latlng_handicap'] = pollsites.apply(lambda x: geocoder.mapbox(x['HANDICAP_ENTRANCE_new'],
                                                                key='INSERT KEY').latlng,axis=1)

Status code 404 from https://api.mapbox.com/geocoding/v5/mapbox.places/2nd Ave (Through walkway / Parking Lot) New York 10029.json: ERROR - 404 Client Error: Not Found for url: https://api.mapbox.com/geocoding/v5/mapbox.places/2nd%20Ave%20(Through%20walkway%20/%20Parking%20Lot)%20New%20York%2010029.json?access_token=pk.eyJ1IjoiaWFtd2Z4IiwiYSI6ImNqN2wyd3dsaDB1aTcycXJyM2s5ZGE1djgifQ.6brfpSfTpw_B_pCNxXCO-A
Status code 404 from https://api.mapbox.com/geocoding/v5/mapbox.places/Essex Street (Through Walkway / Parking Lot) New York 10002.json: ERROR - 404 Client Error: Not Found for url: https://api.mapbox.com/geocoding/v5/mapbox.places/Essex%20Street%20(Through%20Walkway%20/%20Parking%20Lot)%20New%20York%2010002.json?access_token=pk.eyJ1IjoiaWFtd2Z4IiwiYSI6ImNqN2wyd3dsaDB1aTcycXJyM2s5ZGE1djgifQ.6brfpSfTpw_B_pCNxXCO-A
Status code 403 from https://api.mapbox.com/geocoding/v5/mapbox.places/Enter near corner of Adee & Eastchester; take elevator Bronx 10469.json: ERROR - 403 Client Error: Forbid

In [79]:
pollsites['lat'] = pollsites['latlng'].apply(lambda x: x[0])
pollsites['lng'] = pollsites['latlng'].apply(lambda x: x[1])
pollsites['lat_h'] = pollsites['latlng_handicap'].apply(lambda x: x[0] if x is not None else np.nan)
pollsites['lng_h'] = pollsites['latlng_handicap'].apply(lambda x: x[1] if x is not None else np.nan)

In [81]:
voter_sites = pollsites[['BOROUGH','SITE_STATUS','SITE_NAME','SITE_NUMBER','VOTER_ENTRANCE_new','lng','lat']].rename(columns={'VOTER_ENTRANCE_new':'Entrance'})
voter_sites['Entrance_Type'] = 'Voter'
handicap_sites = pollsites[['BOROUGH','SITE_STATUS','SITE_NAME','SITE_NUMBER','HANDICAP_ENTRANCE_new','lng_h','lat_h']].rename(columns={'HANDICAP_ENTRANCE_new':'Entrance',
                                                                                                                                       'lng_h':'lng',
                                                                                                                                        'lat_h':'lat'})
handicap_sites['Entrance_Type'] = 'Handicap'

In [229]:
pollsites_new.shape

(2462, 9)

In [218]:
pollsites_new = pd.concat([voter_sites,handicap_sites])

Join sites to ED 

In [227]:
pollsites_new2 = pd.merge(pollsites_new,ed_pollsite[['SITE_NUMBER','ED']],on='SITE_NUMBER',how='left')

In [191]:
import seaborn as sns
from colormap import rgb2hex

sites = pollsites_new['ED'].unique()
palette = iter(sns.husl_palette(len(sites)))


In [202]:
palette = list(iter(sns.husl_palette(len(sites))))

In [215]:
list(zip(sites,[rgb2hex(int(x[0]*255),int(x[1]*255),int(x[2]*255)) for x in list(palette)]))

[(76, '#F67088'),
 (15, '#F67181'),
 (67, '#F6727A'),
 (16, '#F77272'),
 (12, '#F77369'),
 (14, '#F7745F'),
 (79, '#F77553'),
 (13, '#F77543'),
 (40, '#F67732'),
 (83, '#EF7C32'),
 (43, '#E98031'),
 (41, '#E38431'),
 (86, '#DE8731'),
 (84, '#D98931'),
 (103, '#D58C31'),
 (95, '#D18E31'),
 (2, '#CD9031'),
 (4, '#C99131'),
 (5, '#C59331'),
 (1, '#C29531'),
 (77, '#BE9631'),
 (72, '#BB9731'),
 (113, '#B89931'),
 (73, '#B49A31'),
 (20, '#B19B31'),
 (42, '#AE9C31'),
 (44, '#AA9D31'),
 (37, '#A79E31'),
 (46, '#A39F31'),
 (33, '#A0A131'),
 (6, '#9CA231'),
 (85, '#98A331'),
 (62, '#94A431'),
 (36, '#90A531'),
 (55, '#8CA631'),
 (115, '#87A731'),
 (8, '#82A831'),
 (7, '#7CA931'),
 (9, '#76AB31'),
 (3, '#6EAC31'),
 (75, '#66AD31'),
 (18, '#5CAF31'),
 (66, '#4FB031'),
 (104, '#3EB231'),
 (63, '#31B33B'),
 (32, '#31B24D'),
 (107, '#32B15A'),
 (51, '#32B163'),
 (52, '#32B16B'),
 (17, '#32B072'),
 (53, '#33B078'),
 (24, '#33AF7D'),
 (80, '#33AF81'),
 (49, '#33AF86'),
 (47, '#34AF89'),
 (48, '#34AE8D

In [234]:
pollsites_new.sort_values('SITE_NUMBER').columns

Index(['BOROUGH', 'SITE_STATUS', 'SITE_NAME', 'SITE_NUMBER', 'Entrance', 'lng',
       'lat', 'Entrance_Type', 'geometry'],
      dtype='object')

In [235]:
pollsites_new[['BOROUGH', 'SITE_STATUS', 'SITE_NAME', 'SITE_NUMBER', 'Entrance', 'lng',
       'lat', 'Entrance_Type']].to_csv('day3/Voting_Poll_Sites_new.csv')

In [94]:
voter_sites_geom = gpd.GeoDataFrame(voter_sites,geometry=gpd.points_from_xy(voter_sites.lng, voter_sites.lat),crs="EPSG:4326")

In [99]:
# ed_new = gpd.sjoin(voter_sites_geom[['SITE_NUMBER','geometry']],ed,how='right',op='intersects')

In [150]:
# voter_sites_geom['SITE_NUMBER'] = voter_sites_geom['SITE_NUMBER'].astype(str)
# ed1['SITE_NUMBER'] = ed1['SITE_NUMBER'].astype(str)

In [261]:
ed.rename(columns={'elect_dist_new':'ED'}).to_file('day3/ElectionDistricsNew.geojson',driver='GeoJSON')

In [266]:
site_ED.reset_index()

,SITE_NUMBER,ED
0,00001,"[87, 98, 96]"
1,00007,"[27, 30, 31, 26, 106]"
2,00009,"[2, 3, 5, 4]"
3,00012,"[90, 2, 3, 4, 1, 5, 10, 7, 6, 11, 9, 72, 115, 83]"
4,00013,"[81, 79, 80, 82]"
...,...,...
1226,X0229,"[5, 4]"
1227,X0231,"[12, 13]"
1228,X0234,"[79, 82, 86, 83, 65, 78, 84, 87, 85]"
1229,X0235,[3]


In [275]:
pollsites_new2 = pd.merge(pollsites_new,site_ED.reset_index().rename(columns={'ED':"All_Election_Districts"}),on='SITE_NUMBER')
pollsites_new2[['BOROUGH', 'SITE_STATUS', 'SITE_NAME', 'SITE_NUMBER', 'Entrance', 'lng',
       'lat', 'Entrance_Type', 'All_Election_Districts']].to_csv('day3/Voting_Poll_Sites_new.csv')

In [276]:
pollsites_new2[['BOROUGH', 'SITE_STATUS', 'SITE_NAME', 'SITE_NUMBER', 'Entrance', 'lng',
       'lat', 'Entrance_Type', 'All_Election_Districts']]

,BOROUGH,SITE_STATUS,SITE_NAME,SITE_NUMBER,Entrance,lng,lat,Entrance_Type,All_Election_Districts
0,NaN,A,PS 64 Pura Belpre,X0035,1425 Walton Avenue Bronx 10452,-73.915610,40.840293,Voter,"[76, 15, 67, 16, 12, 14, 79, 13]"
1,NaN,A,PS 64 Pura Belpre,X0035,1425 Walton Avenue Bronx 10452,-73.915610,40.840293,Handicap,"[76, 15, 67, 16, 12, 14, 79, 13]"
2,MANHATTAN,A,JHS 56,11592,220 Henry Street New York 10002,-73.986310,40.713604,Voter,"[40, 83, 43, 41]"
3,MANHATTAN,A,JHS 56,11592,Montgormey Street (NASA Space Double Doors) Ne...,-73.978241,40.756268,Handicap,"[40, 83, 43, 41]"
4,MANHATTAN,A,The River School/PS 281,11485,425 East 35 Street New York 10016,-73.972493,40.744335,Voter,"[86, 84, 103, 95]"
...,...,...,...,...,...,...,...,...,...
2457,BROOKLYN,A,Warbasse Houses Bldg 5C,B0274,West Avenue Brooklyn 11224,-73.972302,40.582739,Handicap,[26]
2458,BRONX,A,Pelham Pkwy Community Center,00036,785 Pelham Parkway North Bronx 10467,-73.863867,40.858142,Voter,[42]
2459,BRONX,A,Pelham Pkwy Community Center,00036,785 Pelham Parkway North - around playgound Br...,-73.863867,40.858142,Handicap,[42]
2460,STATEN IS,A,PS 20 Port Richmond,10574,161 Park Avenue Staten Island 10302,-74.132576,40.636739,Voter,"[21, 23]"


In [240]:
ed1 = ed.merge(ed_pollsite[['ED','SITE_NUMBER']],left_on='elect_dist_new',right_on='ED')